In [2]:
cd /scratch/aqd215/k-gnn/nmr_shift_data/

/scratch/aqd215/k-gnn/nmr_shift_data


In [3]:
import rdkit
import graph_conv_many_nuc_util
import torch
from dataloader import DataLoader
import sys
from torch_geometric.data import (InMemoryDataset, download_url, extract_tar,
                                  Data)
from graph_conv_many_nuc_util import move
from torch.nn import Sequential, Linear, ReLU
import torch.nn.functional as F
from torch_geometric.nn import NNConv
import sys
from loader_processing import process
import loss_functions


class knnGraph(InMemoryDataset):
    def __init__(self,
                 root,
                 transform=None,
                 pre_transform=None,
                 pre_filter=None):
        super(knnGraph, self).__init__(root, transform, pre_transform, pre_filter)
        self.type = type
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return 'temp.pt'

    @property
    def processed_file_names(self):
        return 'whole.pt'
    
    def download(self):
        pass

    def process(self):
        raw_data_list = torch.load(self.raw_paths[0])
        data_list = [
            Data(
                x=d[0],
                edge_index=d[1],
                edge_attr=d[2],
                mask=d[3],
                y=d[4],
                ) for d in raw_data_list
        ]

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

def process(infile):
    dataset_hparams = graph_conv_many_nuc_util.DEFAULT_DATA_HPARAMS
    ds = graph_conv_many_nuc_util.make_datasets({'filename' : infile}, dataset_hparams)
    print('made datasets')
    sys.stdout.flush()

    torch.save(ds, '/scratch/aqd215/k-gnn/nmr_shift_data/temp_files/raw/temp.pt')
    print('saved temp files')
    sys.stdout.flush()

    dataset = knnGraph(root='/scratch/aqd215/k-gnn/nmr_shift_data/temp_files/')
    print('made dataset')
    sys.stdout.flush()
    
    

    train_split = int(len(dataset)*0.6)
    val_split = int(len(dataset)*0.2)
    train_dataset = dataset[:1]
    val_dataset = dataset[1:2]
    test_dataset = dataset[2:3]

    train_loader = DataLoader(train_dataset, batch_size=16, num_workers=1)
    val_loader = DataLoader(val_dataset, batch_size=16, num_workers=1)
    test_loader = DataLoader(test_dataset, batch_size=16, num_workers=1)
    print('created data loaders')
    sys.stdout.flush()

    return train_loader, val_loader, test_loader

In [3]:
infile = '/scratch/aqd215/k-gnn/nmr_shift_data/graph_conv_many_nuc_pipeline.datasets/graph_conv_many_nuc_pipeline.data.13C.nmrshiftdb_hconfspcl_nmrshiftdb.aromatic.64.0.mol_dict.pickle'
                                                               
train_loader, val_loader, test_loader = process(infile)

made datasets
saved temp files
made dataset
created data loaders


In [4]:
for data in train_loader:
    print(data)
    break

Batch(batch=[64], edge_attr=[140, 4], edge_index=[2, 140], mask=[64], x=[64, 37], y=[64])


In [5]:
torch.set_printoptions(profile="full")
data.x

tensor([[6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
         0.],
        [6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
         0.],
        [6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
         0.],
        [6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
         0.],
        [6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
         0.],
        [6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 0.

In [9]:
data.x.size()

torch.Size([64, 37])

In [6]:
data.y

tensor([ 18.3000,  41.8000,  33.5000,  42.2000,  42.0000,  31.7000,  26.5000,
        158.3000, 140.9900,  78.3400,  17.6000, 203.0000,  22.6000,   0.0000,
         33.5000, 193.4000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000])

In [7]:
data.edge_attr

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1.,

In [8]:
data.edge_index

tensor([[ 0,  1,  0,  5,  1,  2,  2,  3,  2, 12,  2, 14,  3,  4,  3,  6,  4,  5,
          4,  9,  4, 10,  6,  7,  7,  8,  8,  9,  8, 15,  9, 11,  9, 13, 11, 16,
         15, 17,  0, 18,  0, 19,  1, 20,  1, 21,  3, 22,  5, 23,  5, 24,  6, 25,
          6, 26,  7, 27, 10, 28, 10, 29, 10, 30, 11, 31, 12, 32, 12, 33, 12, 34,
         13, 35, 14, 36, 14, 37, 14, 38, 15, 39,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  0,  5,  0,  2,  1,  3,  2, 12,  2, 14,  2,  4,  3,  6,  3,  5,  4,
          9,  4, 10,  4,  7,  6,  8,  7,  9,  8, 15,  8, 11,  9, 13,  9, 16, 11,
         17, 15, 18,  0, 19,  0, 20,  1, 21,  1, 22,  3, 23,  5, 24,  5, 25,  6,
         26,  6, 27,  7, 28, 10, 29, 10, 30, 10, 31, 11, 32, 12, 33, 12, 34, 12,
         35, 13, 36, 14, 37, 14, 38, 14, 39

In [8]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        M_in, M_out = 37, 64
        nn1 = Sequential(Linear(4, 128), ReLU(), Linear(128, M_in * M_out))
        self.conv1 = NNConv(M_in, M_out, nn1)

        M_in, M_out = M_out, 64
        nn2 = Sequential(Linear(4, 128), ReLU(), Linear(128, M_in * M_out))
        self.conv2 = NNConv(M_in, M_out, nn2)

        M_in, M_out = M_out, 64
        nn3 = Sequential(Linear(4, 128), ReLU(), Linear(128, M_in * M_out))
        self.conv3 = NNConv(M_in, M_out, nn3)

        self.fc1 = torch.nn.Linear(64, 32)
        self.fc2 = torch.nn.Linear(32, 16)
        self.fc3 = torch.nn.Linear(16, 1)

        self.initialize_weights()

    def forward(self, data):
        x = data.x #4096x37
        x = F.elu(self.conv1(x, data.edge_index, data.edge_attr)) #4096x128
        x = F.elu(self.conv2(x, data.edge_index, data.edge_attr)) #4096x256
        x = F.elu(self.conv3(x, data.edge_index, data.edge_attr)) #4096x512

        x = F.elu(self.fc1(x)) #4096x256
        x = F.elu(self.fc2(x)) #4096x128
        x = self.fc3(x) #4096x1
        return x.flatten() #4096
    
    def initialize_weights(self):
        for m in self.modules():
#             print(m)
            if isinstance(m, Sequential):
                for elem in m:
                    if isinstance(elem, Linear):
                        torch.nn.init.kaiming_uniform_(elem.weight)
            elif isinstance(m, Linear):
                torch.nn.init.kaiming_uniform_(elem.weight)

In [13]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)

def train(epoch):
    model.train()
    loss_all = 0
    total_atoms = 0

    # note that the number of atoms exceeds the number of carbons, and therefore there will be many zeros
    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        atoms = data.mask.sum().item()
        pred = model(data)

        print("mem alloc: {}GB".format(torch.cuda.memory_allocated()*1e-6))
        loss = loss_functions.MSE_loss(pred, data.y, data.mask)
        loss.backward()
        loss_all += loss
        optimizer.step()
        total_atoms += atoms
    return float(loss_all) / total_atoms

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [10]:
def test(loader):
    model.eval()
    error = 0
    loss = 0
    total_atoms = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            atoms = data.mask.sum().item()
            pred = model(data)

            loss += loss_functions.MSE_loss(pred, data.y, data.mask)
            error += loss_functions.MAE_loss(pred, data.y, data.mask)
            total_atoms += atoms

        return float(error) / total_atoms, float(loss) / total_atoms

In [11]:
epochs = []
training_losses = []
val_losses = []
test_losses = []
test_errors = []
for epoch in range(500):
#     torch.cuda.empty_cache()
#     lr = scheduler.optimizer.param_groups[0]['lr']
    lr = 0.001
    avg_train_loss = train(epoch)
    val_error, val_loss = test(val_loader)
#     scheduler.step(val_error)
    test_error, test_loss = test(test_loader)
    print('Epoch: {:03d}, LR: {:7f}, Loss: {:.7f}, Val Loss: {:.7f}, Test Loss: {:.7f}, Test MAE: {:.7f}'.format(epoch, lr, avg_train_loss, val_loss, test_loss, test_error))
    epochs.append(epoch)
    training_losses.append(avg_train_loss)
    val_losses.append(val_loss)
    test_losses.append(test_loss)
    test_errors.append(test_error)

NameError: name 'model' is not defined

In [ ]:
import matplotlib.pyplot as plt
plt.title('Training Loss vs. Epochs (200+)')
plt.plot(epochs[200:], training_losses[200:])
plt.show()

In [ ]:
plt.title('Test Loss vs. Epochs (200+)')
plt.plot(epochs[200:], test_errors[200:])
plt.show()